# Fundamentals of Data Analysis

## Project 2020

In this project perform and explain simple linear regression using Python
on the powerproduction dataset available on Moodle. The goal is to accurately predict wind turbine power output from wind speed values using the data set as a basis.

For enhancement, compare simple linear regression to
other types of regression on this data set

### Project Plan

1. Upload the dataset.
2. Import required libaries for the notebook
3. Intial look at the datset, pandas.describe(), distribution of data, np.random.normal
4. scatterplot of the data using seaborn
5. Explore https://scikit-learn.org/stable/supervised_learning.html#supervised-learning 
6. Understand maybe 5 types of regression on this dataset.
7. Research types of regression that might be suitable - idea of curve/line post initial look at dataset.
8. Regression Vids Tutorials for better understanding:
 - https://www.youtube.com/user/zedstatistics
 - https://en.wikipedia.org/wiki/Regression_analysis
 - https://www.statisticshowto.com/probability-and-statistics/regression-analysis/
 - https://www.sciencedirect.com/topics/medicine-and-dentistry/regression-analysis
9. Plots to describe each regression type
10. subplots for comparison
11. tables for comparison
